# Data Loader

In [1]:
import sys
import math
import mmap
import pickle
import posix_ipc as ipc
import pandas as pd

## 1. Read the pandas data frame

In [2]:
df = pd.read_table("data-01.txt", header=None)
df.head()

,0,1,2,3,4
0,ed0a74c730b1a8470d8dd371714a8b9182d4d860319097...,todayilearned,eug0v,en.wikipedia.org,"<title> jeannette rankin - wikipedia , the fre..."
1,5e67754718366a3da48080bd0f7a9b3ed5f2102ab74218...,todayilearned,eug0v,en.wikipedia.org,<h1> jeannette rankin </h1>
2,142dc63bcecab761cd8273741aeb55c5cfced225ade3d0...,todayilearned,eug0v,en.wikipedia.org,"from wikipedia , the free encyclopedia"
3,18e7c9d00a8ba512e750606f8287236caf94196b102f56...,todayilearned,eug0v,en.wikipedia.org,"jump to : navigation , search"
4,25b5f520a7ff719a45d47c1ca714ba632608d180b9368d...,todayilearned,eug0v,en.wikipedia.org,jeannette rankin


## 2. Store in POSIX inter-process shared memory

In [3]:
storage_name = "/shared-memory-bucket"

In [4]:
def save_to_shared_memory(obj, name):
    size = sys.getsizeof(obj)
    pages = math.ceil(size / ipc.PAGE_SIZE)
    try:
        memory = ipc.SharedMemory(name=name, flags=ipc.O_CREAT, size=ipc.PAGE_SIZE * pages)
    except ValueError: # retry once more
        memory = ipc.SharedMemory(name=name, flags=ipc.O_CREAT, size=ipc.PAGE_SIZE * pages)
    io = mmap.mmap(fileno=memory.fd, length=memory.size, flags=mmap.MAP_SHARED, 
                   prot=mmap.PROT_WRITE)
    io.write(pickle.dumps(obj))
    io.close()
    memory.close_fd()
    return memory.name

In [5]:
save_to_shared_memory(df, storage_name)

'/shared-memory-bucket'

In [6]:
def release_process_from_shared_memory(memory_identifier):
    try:
        shared_mem = ipc.SharedMemory(memory_identifier)
        shared_mem.unlink()
        # will be destructed if every process `unlinked`
        print("INFO: shared memory detached from process")
    except ipc.ExistentialError:
        print("WARN: shared memory not found")
        pass

In [7]:
#release_process_from_shared_memory('/shared-memory-bucket')